In [ ]:
%pip install bitalino
%pip uninstall plux

In [ ]:
%pip install flask

### Test 1

In [ ]:
import matplotlib.pyplot as plt

def updatePlot(points):
    """
    Met à jour un plot avec la liste de points fournie.
    Si aucun plot n'existe, en crée un.
    Les points sont reliés par des lignes.
    
    points : liste de tuples (x, y)
    """
    # Vérifie si une figure est déjà ouverte
    if not plt.get_fignums():
        plt.ion()  # mode interactif pour mise à jour dynamique
        fig, ax = plt.subplots()
        ax.set_title("Dynamic Plot")
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.grid(True)
        line, = ax.plot([], [], 'o-', color='blue')  # points reliés par des traits
    else:
        fig = plt.gcf()
        ax = plt.gca()
        line = ax.lines[0] if ax.lines else ax.plot([], [], 'o-', color='blue')[0]

    # Sépare les points en x et y
    x_vals = [p[0] for p in points]
    y_vals = [p[1] for p in points]

    # Met à jour les données de la ligne
    line.set_data(x_vals, y_vals)
    ax.relim()        # recalcul des limites
    ax.autoscale()    # ajustement automatique des axes
    plt.draw()
    plt.pause(0.01)


import platform
import sys

osDic = {
    "Darwin": f"MacOS/Intel{''.join(platform.python_version().split('.')[:2])}",
    "Linux": "Linux64",
    "Windows": f"Win{platform.architecture()[0][:2]}_{''.join(platform.python_version().split('.')[:2])}",
}
if platform.mac_ver()[0] != "":
    import subprocess
    from os import linesep

    p = subprocess.Popen("sw_vers", stdout=subprocess.PIPE)
    result = p.communicate()[0].decode("utf-8").split(str("\t"))[2].split(linesep)[0]
    if result.startswith("12."):
        print("macOS version is Monterrey!")
        osDic["Darwin"] = "MacOS/Intel310"
        if (
            int(platform.python_version().split(".")[0]) <= 3
            and int(platform.python_version().split(".")[1]) < 10
        ):
            print(f"Python version required is ≥ 3.10. Installed is {platform.python_version()}")
            exit()


sys.path.append(f"PLUX-API-Python3/{osDic[platform.system()]}")

import plux

print(plux.__file__)

class NewDevice(plux.SignalsDev):
    

    def __init__(self, address):
        plux.SignalsDev.__init__(address)
        self.duration = 0
        self.frequency = 0
        self.ListP=[]

    def onRawFrame(self, nSeq, data):  # onRawFrame takes three arguments
        print(nSeq, *data)
        self.ListP.append([nSeq,data[0]])
        if (nSeq%100 == 0 ):
            updatePlot(self.ListP[100:])
        return nSeq > self.duration * self.frequency



# example routines


def exampleAcquisition(
    address="98:D3:C1:FE:04:BB",
    duration=20,
    frequency=1000,
    active_ports=[1, 2, 3, 4, 5, 6],
):  # time acquisition for each frequency
    """
    Example acquisition.
    """
    device = NewDevice(address)
    device.duration = int(duration)  # Duration of acquisition in seconds.
    device.frequency = int(frequency)  # Samples per second.
    
    # Trigger the start of the data recording: https://www.downloads.plux.info/apis/PLUX-API-Python-Docs/classplux_1_1_signals_dev.html#a028eaf160a20a53b3302d1abd95ae9f1
    device.start(device.frequency, active_ports, 16)
    device.loop()  # calls device.onRawFrame until it returns True
    device.stop()
    device.close()


if __name__ == "__main__":
    # Use arguments from the terminal (if any) as the first arguments and use the remaining default values.
    exampleAcquisition()

# Backend

In [1]:
import platform
import sys
import threading
import queue
from flask import Flask, render_template, Response

# --- Détection OS pour PLUX ---
osDic = {
    "Darwin": f"MacOS/Intel{''.join(platform.python_version().split('.')[:2])}",
    "Linux": "Linux64",
    "Windows": f"Win{platform.architecture()[0][:2]}_{''.join(platform.python_version().split('.')[:2])}",
}

if platform.mac_ver()[0] != "":
    import subprocess
    from os import linesep

    p = subprocess.Popen("sw_vers", stdout=subprocess.PIPE)
    result = p.communicate()[0].decode("utf-8").split(str("\t"))[2].split(linesep)[0]
    if result.startswith("12."):
        print("macOS version is Monterrey!")
        osDic["Darwin"] = "MacOS/Intel310"
        if int(platform.python_version().split(".")[0]) <= 3 and int(platform.python_version().split(".")[1]) < 10:
            print(f"Python version required is ≥ 3.10. Installed is {platform.python_version()}")
            exit()

sys.path.append(f"PLUX-API-Python3/{osDic[platform.system()]}")
import plux
print("PLUX API path:", plux.__file__)

# --- Queue pour SSE ---
data_queue = queue.Queue()

# --- Classe BITalino utilisant onRawFrame ---
class NewDevice(plux.SignalsDev):
    def __init__(self, address):
        plux.SignalsDev.__init__(self, address)  # passe self explicitement
        self.duration = 0
        self.frequency = 0

    def onRawFrame(self, nSeq, data):
        # Envoi vers la queue
        data_queue.put({"seq": nSeq, "value": data[0]})
        # Retourne True si acquisition terminée
        return nSeq >= self.duration * self.frequency

# --- Fonction acquisition ---
def exampleAcquisition(
    address="98:D3:C1:FE:04:BB",
    duration=20,
    frequency=1000,
    active_ports=None
):
    if active_ports is None:
        active_ports = [1, 2, 3, 4, 5, 6]

    device = NewDevice(address)
    device.duration = int(duration)
    device.frequency = int(frequency)

    try:
        device.start(device.frequency, active_ports, 16)
        device.loop()  # appelle onRawFrame jusqu'à ce qu'elle retourne True
    except Exception as e:
        print("Erreur BITalino:", e)
    finally:
        device.stop()
        device.close()

# --- Flask ---
app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")

def event_stream():
    while True:
        data = data_queue.get()
        yield f"data: {data}\n\n"

@app.route("/stream")
def stream():
    return Response(event_stream(), mimetype="text/event-stream")

# --- Lancement acquisition en thread ---
if __name__ == "__main__":
    threading.Thread(target=exampleAcquisition, daemon=True).start()
    app.run(debug=True, threaded=True)

PLUX API path: c:\Users\doero\OneDrive\Bureau\Backup\COURS 2025-2026\Facteur Humain\ProjetPython1\python-samples\PLUX-API-Python3/Win64_39\plux.pyd
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\doero\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\doero\AppData\Local\Programs\Python\Python39\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\doero\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\doero\AppData\Local\Programs\Python\Python39\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\doero\AppData\Local\Temp\ipykernel_13792\4187778657.py", line 57, in exampleAcquisition
  File "C:\Users\doero\AppData\Local\Temp\ipykernel_13792\4187778657.py", line 37, in __init__
TypeError: object.__init__() takes exactly one argument (the instance to initialize)
